## <span style="color: rgb(34, 34, 34); font-family: -system-ui, -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Roboto, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; font-size: 20px; font-weight: bold; text-align: center;">Ingredient Optimisation</span>

### **1. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">What are the standard ingredients for each pizza?</span>**

<span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">Ans - The challenge in this question is that toppings for each pizza are stored like (</span>1, 2, 3, 4, 5, 6, 8, 10<span style="color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; caret-color: rgb(64, 64, 64);">). We need to seggregate it in order to use it. So for that I have used <b>STRING_SPLIT() clause </b>which breaks the given column into different rows based on the separator.&nbsp;<br>Then I trimmed it result as it had leading and trailing spaces. Casted it to int so that it can be used in conjuction with other tables.&nbsp;<br>(<span style="background-color: yellow;">Note :-</span> Compatibility(version) of sql should be equal to or greater than&nbsp;</span> <span style="font-size: inherit; font-family: inherit; color: rgb(64, 64, 64); font-style: inherit; font-variant-caps: inherit; caret-color: rgb(64, 64, 64);">130.)</span>

In [1]:
with pizza_toppings_seggregated as(
select cast(trim(value) as int) as topping,count(cast(trim(value) as int)) as topping_count
from pizza_recipes
cross apply string_split(pizza_recipes.toppings, ',') 
GROUP BY cast(trim(value) as int)
)
select topping_name from pizza_toppings a inner join pizza_toppings_seggregated b on a.topping_id = b.topping where topping_count > 1

(2 rows affected)

Total execution time: 00:00:00.020

topping_name
Cheese
Mushrooms


Things learnt - 

- STRING\_SPLIT()

### **2. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">What was the most commonly added extra?</span>**

<span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">Ans - Applied the same concept of STRING_SPLIT() used above here. Selected the non null and non empty values and passed the result as an subquery to pizza_toppings table to find the topping name.</span>

In [6]:
with not_null_customer_orders as(
select * from customer_orders where extras <> ''
)
select * from pizza_toppings where topping_id = (select top 1 cast(trim(value) as int) from not_null_customer_orders 
cross apply string_split(not_null_customer_orders.extras, ',') where extras <> 'null' group by cast(trim(value) as int) order by count(cast(trim(value) as int)) desc)

(1 row affected)

Total execution time: 00:00:00.022

topping_id,topping_name
1,Bacon


### **3. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">What was the most common exclusion?</span>**

<span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">Ans - Similar to above question. Just changed extras to exclusions.&nbsp;</span>

In [5]:
with not_null_customer_orders as(
select * from customer_orders where exclusions <> ''
)
select * from pizza_toppings where topping_id = (select top 1 cast(trim(value) as int) from not_null_customer_orders 
cross apply string_split(not_null_customer_orders.exclusions, ',') where exclusions <> 'null' group by cast(trim(value) as int) order by count(cast(trim(value) as int)) desc)

(1 row affected)

Total execution time: 00:00:00.040

topping_id,topping_name
4,Cheese


### **4. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">Generate an order item for each record in the</span> `customers_orders` <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">table in the format of one of the following:</span>**

1. - `Meat Lovers`
    - `Meat Lovers - Exclude Beef`
    - `Meat Lovers - Extra Bacon`
    - `Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers`

### **5. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">Generate an alphabetically ordered comma separated ingredient list for each pizza order from the</span> `customer_orders` <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">table and add a</span> `2x` <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">in front of any relevant ingredients</span>**

1. - For example: `"Meat Lovers: 2xBacon, Beef, ... , Salami"`

### **6. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?</span>**

Ans - So I went all in for CTE's in this question. Although I could have avoided 2,3 blocks in here but I choose not to, in order to not compromise readibility.   
The query writteen below is quite simple and nothing fancy is used.

In [9]:
with sucessful_customer_orders as(
select b.order_id,pizza_id,extras,exclusions from orders_delivered a inner join customer_orders b on a.order_id = b.order_id
),
not_null_customer_orders as(
select * from sucessful_customer_orders where extras <> ''
),
extras_count as(
select cast(trim(value) as int) as extras_id,count(cast(trim(value) as int)) as count_extras_id from not_null_customer_orders 
cross apply string_split(not_null_customer_orders.extras, ',') where extras <> 'null' group by cast(trim(value) as int)
),
not_null_customer_orders_exclusions as(
select * from sucessful_customer_orders where exclusions <> ''
),
exclusion_count as(
select cast(trim(value) as int) as exclusion_id,count(cast(trim(value) as int)) as count_exclusions_id from not_null_customer_orders_exclusions 
cross apply string_split(not_null_customer_orders_exclusions.exclusions, ',') where exclusions <> 'null' group by cast(trim(value) as int)
),
new_table as(
select topping_id,topping_name,isnull(count_exclusions_id,0) as count_exclusions, isnull(count_extras_id,0) as count_extras from exclusion_count a right join pizza_toppings b on a.exclusion_id = b.topping_id 
left join extras_count c on b.topping_id = c.extras_id
)
select topping_id,topping_name,(count_extras-count_exclusions) as frequency from new_table order by count_extras-count_exclusions desc

(12 rows affected)

Total execution time: 00:00:00.032

topping_id,topping_name,frequency
1,Bacon,3
3,Beef,0
7,Onions,0
8,Pepperoni,0
9,Peppers,0
10,Salami,0
11,Tomatoes,0
12,Tomato Sauce,0
5,Chicken,0
6,Mushrooms,-1
